In [32]:
#import libraries
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [33]:
MODEL = "llama3.2"


In [34]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

        
        
    

In [35]:
# Let's try one out
ed = Website("https://cnn.com")
print(ed.title)
print(ed.text)




Breaking News, Latest News and Videos | CNN
CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
More
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Watch
Listen
Live TV
Subscribe
Sign in
My Account
Settings
Newsletters
Topics You Follow
Sig

In [36]:
#Define our system prompt 
system_prompt = "You are an assistant that analyzes the contents of a website\
and provides a short summary,ignoring text that might be navigation related \
Respond in markdown carfully and organize"

In [37]:
def user_prompt_for(website):
    user_prompt = f"You are looking at the website titled: {website.title}\n\n" 
    user_prompt += "The contents of this website are as follows:\n"
    user_prompt += "Please provide a short summary of this website in markdown.\n"
    user_prompt += "If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text 
    return user_prompt


In [38]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [39]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [40]:
# And now: call the Ollama function

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [41]:
def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))

In [42]:
display_summary("https://cnn.com")

Here are some headlines and summaries from the article:

**Business**

* "A Chinese EV giant is now offering free driver assistance tech on cars under $10,000"
* "Excessive regulation could 'kill' AI industry, JD Vance tells government leaders at Paris summit"

**Technology**

* "Sam Altman fires back at Elon Musk: 'I don't think he's a happy person. I feel for him'"
* "Axel Schmidt/Reuters"
* "A Chinese EV giant is now offering free driver assistance tech on cars under $10,000"

**Health**

* "Marijuana ads are enticing kids to try weed, study says"
* "Experts tell parents how to help their kids through their anxiety"
* "Dermatologists share how to treat dry skin"

**Science**

* "Experimental transplants with organs from gene-edited pigs see major steps forward"
* "First words decoded from burnt scrolls that haven’t been read in nearly 2,000 years"

**Space**

* "The US dodged a bird flu pandemic in 1957 thanks to eggs and luck. Can it happen again?"

These are just some of the headlines and summaries provided. If you'd like me to extract more information or summarize specific topics, please let me know!